### How to read a csv, filter what you want and write a new csv

In [87]:
from pandas import *
from pathlib import Path
from requests import get
from io import StringIO

### read CSV file

In [99]:
'''
in_path = Path(r'/Users/ehaas/Documents/FHIR/synthea/output/csv/') 
r_type = "encounters"
csv_file = in_path / f'{r_type}.csv'
'''
csv_file = Path(r'/Users/ehaas/Downloads/4717cc76502dbe8776555d0be9f19bde-963c2881815d1c9c06c9c157b1c602dfceb322ec/OMH DataPoint Element to FHIR Element Mapping Table.csv')
df = read_csv(csv_file, skiprows=1)
df.head()

,blood_glucose,laboratory,http://loinc.org,2339-0,Glucose Mass/volume in Blood
0,blood_pressure,vital-signs,http://loinc.org,85354-9,NA*
1,body_temperature,vital-signs,http://loinc.org,8310-5,Body temperature
2,body_weight,vital-signs,http://loinc.org,29463-7,Body weight
3,heart_rate,vital-signs,http://loinc.org,8867-4,Heart rate
4,oxygen_saturation,vital-signs,http://loinc.org,59408-5,Oxygen saturation in Arterial blood by Pulse o...


### read Excel file (this reads only the first sheet)

In [102]:
my_file = r'/Users/ehaas/Downloads/4717cc76502dbe8776555d0be9f19bde-963c2881815d1c9c06c9c157b1c602dfceb322ec/book2.xlsx'
excel_file = Path(my_file)
print(excel_file)
df = read_excel(excel_file)
df.head()

/Users/ehaas/Downloads/4717cc76502dbe8776555d0be9f19bde-963c2881815d1c9c06c9c157b1c602dfceb322ec/book2.xlsx


,Header.schema_id.name,Observation.category.code,Observation.code.system,Observation.code.code,Observation.code.display
0,blood_glucose,laboratory,http://loinc.org,2339-0,Glucose Mass/volume in Blood
1,blood_pressure,vital-signs,http://loinc.org,85354-9,NA*
2,body_temperature,vital-signs,http://loinc.org,8310-5,Body temperature
3,body_weight,vital-signs,http://loinc.org,29463-7,Body weight
4,heart_rate,vital-signs,http://loinc.org,8867-4,Heart rate


### Get all sheets from Excel file

In [153]:
#my_file = r'/Users/ehaas/Downloads/4717cc76502dbe8776555d0be9f19bde-963c2881815d1c9c06c9c157b1c602dfceb322ec/book2.xlsx'
my_file = r'/Users/ehaas/Documents/FHIR/OMHtoFHIR/mapping/mapping tables/json-schema-fhir-mappings.xlsx'
excel_wb_file = Path(my_file)
df = read_excel(excel_wb_file, sheet_name=None)
df.keys() # = sheet names

odict_keys(['omh-header-fhir-observation-map', 'ieee-header-fhir-observation-ma', 'ieee-header-fhir-provenance-map', 'omh-header-fhir-provenance-mapp', 'omh-heartrate-fhir-observation-', 'omh-resprate-fhir-observation-m', 'omh-bodytemp-fhir-observation-m', 'omh-bloodpressure-fhir-observat', 'omh-bodyweight-fhir-observation', 'omh-sleeptime-fhir-observation-', 'omh-stepcount-fhir-observation-', 'omh-bloodglucose-fhir-observati', 'omh-o2sat-fhir-observation-mapp'])

,Foo,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,NaN,NaN,Bar,NaN
1,Header.schema_id.name,Observation.category.code,Observation.code.system,Observation.code.code,Observation.code.display
2,blood_glucose,laboratory,http://loinc.org,2339-0,Glucose Mass/volume in Blood
3,blood_pressure,vital-signs,http://loinc.org,85354-9,NA*
4,body_temperature,vital-signs,http://loinc.org,8310-5,Body temperature
5,body_weight,vital-signs,http://loinc.org,29463-7,Body weight
6,heart_rate,vital-signs,http://loinc.org,8867-4,Heart rate
7,oxygen_saturation,vital-signs,http://loinc.org,59408-5,Oxygen saturation in Arterial blood by Pulse o...
8,respiratory_rate,vital-signs,http://loinc.org,9279-1,Respiratory Rate
9,sleep_duration,physical-activity,http://snomed.info/id,248263006,Duration of sleep (observable entity)


## reassign columns

In [151]:
new_headers = df['Sheet1'].iloc[1]
df2 = df['Sheet1'][2:]
df2.columns = new_headers
#df2.reset_index(drop=True,, inplace=True)
df2

1,Header.schema_id.name,Observation.category.code,Observation.code.system,Observation.code.code,Observation.code.display
2,blood_glucose,laboratory,http://loinc.org,2339-0,Glucose Mass/volume in Blood
3,blood_pressure,vital-signs,http://loinc.org,85354-9,NA*
4,body_temperature,vital-signs,http://loinc.org,8310-5,Body temperature
5,body_weight,vital-signs,http://loinc.org,29463-7,Body weight
6,heart_rate,vital-signs,http://loinc.org,8867-4,Heart rate
7,oxygen_saturation,vital-signs,http://loinc.org,59408-5,Oxygen saturation in Arterial blood by Pulse o...
8,respiratory_rate,vital-signs,http://loinc.org,9279-1,Respiratory Rate
9,sleep_duration,physical-activity,http://snomed.info/id,248263006,Duration of sleep (observable entity)
10,step_count,physical-activity,http://loinc.org,55423-8,Number of steps in unspecified time Pedometer


### save as CSV

In [152]:
new_file_name = f"{list(df.keys())[0]}.csv"
new_path = r'/Users/ehaas/Documents/Python/MyNotebooks/Panda_utils/CSV/'
new_file = Path(new_path)/new_file_name 
df2.to_csv(new_file, index=False)
print(f'printing {new_file_name} to {new_file}')

printing Sheet1.csv to /Users/ehaas/Documents/Python/MyNotebooks/Panda_utils/CSV/Sheet1.csv


### Downloading the csv file from your GitHub account

In [71]:
url = "https://raw.githubusercontent.com/Healthedata1/mFHIR/master/mapping/mapping%20tables/omh-sleeptime-fhir-observation-mapping.csv" # Make sure the url is the raw version of the file on GitHub
download = get(url).content


### Reading the downloaded content and turning it into a pandas dataframe

In [72]:
df = read_csv(StringIO(download.decode('utf-8')), skiprows=1)

### Printing out the first 5 rows of the dataframe ( skip the first row ) 

In [85]:
df.head

<bound method NDFrame.head of               Unnamed: 0                                         Unnamed: 1  \
0                    NaN                                                NaN   
1   OMH Total Sleep Time                                                NaN   
2                  Index                       OMH Total Sleep Time Element   
3                    ---                                                ---   
4                      1                          total-sleep-time-1.0.json   
5                    2.1                                   total_sleep_time   
6                    2.2                             total_sleep_time.value   
7                    2.3                              total_sleep_time.unit   
8                    2.4                              total_sleep_time.unit   
9                    2.5                              total_sleep_time.unit   
10                     3                          body.effective_time_frame   
11                    

#### Read csv as panda dataframe

In [10]:
df = read_csv(csv_file)
df.head()

,header.schema_id.name,Observation.category.code,Observation.code.system,Observation.code.code,Observation.code.display
0,blood_glucose,laboratory,http://loinc.org,2339-0,Glucose Mass/volume in Blood
1,blood_pressure,vital-signs,http://loinc.org,85354-9,NA*
2,body_temperature,vital-signs,http://loinc.org,8310-5,Body temperature
3,body_weight,vital-signs,http://loinc.org,29463-7,Body weight
4,heart_rate,vital-signs,http://loinc.org,8867-4,Heart rate


#### filter what you want ...

- choose only encounters of type [1234,261665006]

In [11]:
df.columns

Index(['header.schema_id.name', 'Observation.category.code',
       'Observation.code.system', 'Observation.code.code',
       'Observation.code.display'],
      dtype='object')

In [12]:
df

,header.schema_id.name,Observation.category.code,Observation.code.system,Observation.code.code,Observation.code.display
0,blood_glucose,laboratory,http://loinc.org,2339-0,Glucose Mass/volume in Blood
1,blood_pressure,vital-signs,http://loinc.org,85354-9,NA*
2,body_temperature,vital-signs,http://loinc.org,8310-5,Body temperature
3,body_weight,vital-signs,http://loinc.org,29463-7,Body weight
4,heart_rate,vital-signs,http://loinc.org,8867-4,Heart rate
5,oxygen_saturation,vital-signs,http://loinc.org,59408-5,Oxygen saturation in Arterial blood by Pulse o...
6,respiratory_rate,vital-signs,http://loinc.org,9279-1,Respiratory Rate
7,sleep_duration,physical-activity,http://snomed.info/id,248263006,Duration of sleep (observable entity)
8,step_count,physical-activity,http://loinc.org,55423-8,Number of steps in unspecified time Pedometer


### filter by data

In [25]:
schema_name = 'blood_glucose'
filter_1 = df['header.schema_id.name'] == schema_name
cat_name = 'laboratory'
filter_2 = df['Observation.category.code'] == cat_name
my_filter = filter_1 & filter_2
my_filter


0     True
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
dtype: bool

In [16]:
df[my_filter]

,header.schema_id.name,Observation.category.code,Observation.code.system,Observation.code.code,Observation.code.display
0,blood_glucose,laboratory,http://loinc.org,2339-0,Glucose Mass/volume in Blood


In [19]:
df[my_filter]['Observation.category.code']

0    laboratory
Name: Observation.category.code, dtype: object

In [22]:
df[my_filter]['Observation.category.code'].iloc[0]

'laboratory'

In [ ]:
df.shape

In [ ]:
is_code_filter = df.CODE.isin([1234,261665006])

print(type(is_code_filter))

is_code_filter

In [ ]:
df[is_code_filter]

#### remove stop date for unknown encounter code types to create in-progress encounters

In [ ]:
df.loc[df.CODE==1234, 'STOP'] = None


df[is_code_filter]

#### overwrite csv for encounters

In [ ]:
new_csv_file = in_path / f'{r_type}.csv'
df[is_code_filter].to_csv(new_csv_file)

### delete selected files  ( only simple encounter for now )

In [ ]:
rem_files =[
'allergies.csv',
'careplans.csv',
'imaging_studies.csv',
'immunizations.csv',
'medications.csv',
'observations.csv',
]

for rem_file in rem_files:
    rem_path = in_path / rem_file
    try:
        rem_path.unlink()
    except FileNotFoundError:
            pass

### make subdir amd move all csv to it. directory to save

In [ ]:
new_dir = input('What is new directory name?')

In [ ]:
out_path = in_path / new_dir
print(f'out_path = {out_path}')
out_path.mkdir()
for source in in_path.glob("*.csv"):
    print(f'source = {source}, source.name = {source.name}')
    destination = out_path / source.name
    print(f'destination = {destination}')
    source.replace(destination)